In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from skimage import color
from time import time
from scipy import signal
import imageio

In [ ]:
img = plt.imread('/content/drive/MyDrive/COL783/1024px-Broadway_tower_edit.jpg')

In [ ]:
plt.imshow(img)

In [ ]:
def energyfunction(img): 
  img1 = np.copy(img)
  img1 = color.rgb2gray(img1)
  #finding the gradient in x and y dir
  sobelx = cv2.Sobel(img1,cv2.CV_64F,1,0,ksize=5)
  sobely = cv2.Sobel(img1,cv2.CV_64F,0,1,ksize=5)
  #energy function
  energy = abs(sobelx) + abs(sobely)
  return energy

def plotseam(img,cols):
  img1 = np.copy(img)
  for i in range(img.shape[0]):
    indx = img.shape[0] -1 - i
    img1[indx,cols[i]] = [255,0,0]
  #plt.imshow(img1)
  #plt.show()
  return img1

def plotseam1(img,cols):
  img1 = np.copy(img)
  for i in range(img.shape[0]):
    indx = img.shape[0] -1 - i
    img1[indx,1+cols[i]] = [255,0,0]
  #plt.imshow(img1)
  #plt.show()
  return img1

def findSeam(energy):
  dp = np.copy(energy)
  paths = np.zeros(dp.shape)
  for i in range(1,dp.shape[0]):
    for j in range(dp.shape[1]):
      if (j>=1 and j+1<dp.shape[1]):
        dp[i,j] = dp[i,j] + np.min([dp[i-1,j-1],dp[i-1,j],dp[i-1,j+1]])
        paths[i,j] = np.int(np.argmin([dp[i-1,j-1],dp[i-1,j],dp[i-1,j+1]]))
      elif (j+1>=dp.shape[1]):
        dp[i,j] = dp[i,j] + np.min([dp[i-1,j-1],dp[i-1,j]])
        paths[i,j] = np.int(np.argmin([dp[i-1,j-1],dp[i-1,j]]))
      elif (j<1):
        dp[i,j] = dp[i,j] + np.min([dp[i-1,j],dp[i-1,j+1]])
        paths[i,j] = np.int(1 + np.argmin([dp[i-1,j],dp[i-1,j+1]]))

  #now we have cum energy, now we have to find the min seam
  cols = []
  col = np.argmin(dp[dp.shape[0]-1,:])
  cols.append(col)
  for i in range(dp.shape[0]-1):
    indx = dp.shape[0] - 1 - i
    col = np.int(col + paths[indx,col] - 1)
    cols.append(col)

  return cols

def deleteseam(img,cols):
  img1 = np.zeros((img.shape[0],img.shape[1]-1,3))
  for i in range(img1.shape[0]):
    indx = img1.shape[0] - 1 - i
    col = cols[i]
    img1[indx,:col] = img[indx,:col]
    if (1+col < img.shape[1]):
      img1[indx,col:] = img[indx,1+col:]
  
  return img1.astype('uint8')

def insertseam(img,cols):
  img1 = np.zeros((img.shape[0],img.shape[1]+1,3))
  img = img.astype('float32')
  img1 = img1.astype('float32')
  for i in range(img.shape[0]):
    indx = img.shape[0]-i-1
    col = cols[i]
    img1[indx,:1+col] = img[indx,:1+col]

    if (col-1>=0 and col+1<img.shape[1]):
      img1[indx,1+col,0] = np.int((img[indx,col-1][0] + img[indx,col+1,0])/2)
      img1[indx,1+col,1] = np.int((img[indx,col-1][1] + img[indx,col+1,1])/2)
      img1[indx,1+col,2] = np.int((img[indx,col-1][2] + img[indx,col+1,2])/2)
    elif (col-1 < 0):
      img1[indx,1+col] = img[indx,col+1]
    elif (col+1 >= img.shape[1]):
      img1[indx,1+col] = img[indx,col-1]
    
    if (1+col<img.shape[1]):
      img1[indx,col+2:] = img[indx,col+1:]
    
  return img1.astype('uint8')

In [ ]:
def enlargevert(img,dy):
  img_list_video = []
  img1 = np.copy(img)
  #add dy vertical seams
  seam_list = []
  #first find dy seams
  #store the seams in seam_list
  for i in range(dy):
    energy = energyfunction(img1)
    cols = findSeam(energy)
    seam_list.append(cols)
    img1 = deleteseam(img1,cols)  

  #now insert seam
  img2 = np.copy(img)
  for i in range(dy):
    img2 = insertseam(img2,seam_list[i])
    img_list_video.append(img2)
    img3 = plotseam(img2,seam_list[i])
    img_list_video.append(img3)

    '''
    print(img2.shape)
    plt.imshow(img2)
    plt.show()
    plt.imshow(img3)
    plt.show()
    '''

  img4 = np.copy(img2)
  for i in range(dy):
    img4 = plotseam1(img4,seam_list[i])
  

  return img2,img_list_video,img4


In [ ]:
def enlarge(img,dx,dy):
  img_list = []
  #add vertical seams
  img1 = np.copy(img)
  img2,images,seamimg = enlargevert(img1,dy)
  img_list.append(images)

  plt.figure(figsize=(10,10))
  plt.title('Image with added seams')
  plt.imshow(seamimg)
  plt.show()

  #add horizontal seams
  img3 = np.copy(img2)
  img3 = np.rot90(img3)
  img4,images,seamimg = enlargevert(img3,dx)

  seamimg = np.rot90(seamimg)
  seamimg = np.rot90(seamimg)
  seamimg = np.rot90(seamimg)

  plt.figure(figsize=(10,10))
  plt.title('Image with added seams')
  plt.imshow(seamimg)
  plt.show()

  img4 = np.rot90(img4)
  img4 = np.rot90(img4) 
  img4 = np.rot90(img4)

  img_list.append(images)

  return img4,img_list



In [ ]:
img1 = np.copy(img)
start = time()
img2,img_list = enlarge(img1,20,20)
end = time()
print('Time elapses for normal method: ',(end - start)/60,'minutes')

In [ ]:
plt.imshow(img2)
print(img2.shape)

In [ ]:
#for video
dx = 2
dy = 2
image_list = []
for i in range(len(img_list[0])):
    image_list.append(img_list[0][i])
    plt.imshow(img_list[0][i])
    plt.show()

In [ ]:
for i in range(len(img_list[1])):
  im1 = img_list[1][i]
  im1 = np.rot90(im1)
  im1 = np.rot90(im1)
  im1 = np.rot90(im1)
  image_list.append(im1)
  plt.imshow(im1)
  plt.show()

In [ ]:
imageio.mimsave('seam_enlarge.gif',image_list)

In [ ]:
#enlarge 
#3  horizontal seam
dx = 5
#2 vertical seams
dy = 2
print(dx,dy)

In [ ]:
#fast seam
start = time()
!python3 "/content/drive/MyDrive/COL783/main.py" 20 20
end = time()
print('Time elapsed for fast seam: ',end - start,'seconds')

In [ ]:
result = plt.imread('/content/drive/MyDrive/COL783/fastseamenlarge.jpg')
plt.figure(figsize=(10,10))
plt.imshow(result)
print('The size of the image: ',result.shape)